In [1]:
import sys
sys.path.append('../py_scripts')

In [2]:
import keras
import os
from model_utils import get_mobilenet_model, get_callbacks
from utils import get_train_test_data_dict, DictImageDataGenerator, preprocess_func
from utils import get_if_new_whale_dict, split_train_test_dict

# define train params
IMG_SIZE = 224
LABEL_CNT = 2
ALL_DATA_JSON = '../data/train_data.json'
BATCH_SIZE = 8
ALL_DATA_DICT = json.loads(open(ALL_DATA_JSON).read())
print('load done')

if_whale_d = get_if_new_whale_dict(ALL_DATA_DICT)
train_d, val_d = split_train_test_dict(if_whale_d)
print(len(train_d['new_whale']))
print(len(val_d['new_whale']))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


load done
7732
1932


In [3]:
train_ds = DictImageDataGenerator(rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  preprocessing_function=preprocess_func)
train_gen = train_ds.flow_from_dict(train_d, target_size=(
    IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)
val_ds = DictImageDataGenerator(preprocessing_function=preprocess_func)
val_gen = val_ds.flow_from_dict(val_d, target_size=(
    IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE)
val_steps = val_gen.samples//BATCH_SIZE
print(val_steps)
# test
for x, y in train_gen:
    print(x.shape, y.shape)
    break
print(train_gen.class_indices)

Found 20290 images belonging to 2 classes.
Found 5071 images belonging to 2 classes.
633
(8, 224, 224, 3) (8, 2)
{'new_whale': 0, 'not_new_whale': 1}


In [4]:
mob_model = get_mobilenet_model(IMG_SIZE, LABEL_CNT, dense_dim=64)
mob_model.summary()

/usr/local/lib/python3.5/dist-packages/keras_applications/mobilenet.py:206: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [5]:
cb_list = get_callbacks('../data/checkpoints/detect_if_new_whale.h5', mob_model)
adam_opt = keras.optimizers.Adam(lr=0.001)
mob_model.compile(optimizer=adam_opt,
                  loss='categorical_crossentropy', metrics=['acc'])
print('compile done')

load pre weights
compile done


In [6]:
# train
mob_model.fit_generator(
    train_gen,
    steps_per_epoch=2000,
    epochs=100,
    verbose=1,
    callbacks=cb_list,
    validation_data=val_gen,
    validation_steps=val_steps
)

Epoch 1/100
2000/2000 [==============================] - 1849s 924ms/step - loss: 4.3577 - acc: 0.7237 - val_loss: 4.0798 - val_acc: 0.7409

Epoch 00001: val_loss improved from inf to 4.07979, saving model to ../data/checkpoints/detect_if_new_whale.h5
Epoch 2/100
2000/2000 [==============================] - 1772s 886ms/step - loss: 4.4680 - acc: 0.7149 - val_loss: 4.3990 - val_acc: 0.7210

Epoch 00002: val_loss did not improve from 4.07979
Epoch 3/100
2000/2000 [==============================] - 1783s 892ms/step - loss: 3.9597 - acc: 0.7217 - val_loss: 1.9783 - val_acc: 0.6936

Epoch 00003: val_loss improved from 4.07979 to 1.97825, saving model to ../data/checkpoints/detect_if_new_whale.h5
Epoch 4/100
2000/2000 [==============================] - 1782s 891ms/step - loss: 1.2917 - acc: 0.7000 - val_loss: 1.1692 - val_acc: 0.7363

Epoch 00004: val_loss improved from 1.97825 to 1.16916, saving model to ../data/checkpoints/detect_if_new_whale.h5
Epoch 5/100
2000/2000 [=====================

2000/2000 [==============================] - 1816s 908ms/step - loss: 0.3771 - acc: 0.8341 - val_loss: 0.5432 - val_acc: 0.7736

Epoch 00039: val_loss did not improve from 0.48106
Epoch 40/100
2000/2000 [==============================] - 1813s 907ms/step - loss: 0.3819 - acc: 0.8324 - val_loss: 0.5264 - val_acc: 0.7801

Epoch 00040: val_loss did not improve from 0.48106
Epoch 41/100
2000/2000 [==============================] - 1819s 909ms/step - loss: 0.3727 - acc: 0.8388 - val_loss: 0.5277 - val_acc: 0.7789

Epoch 00041: val_loss did not improve from 0.48106
Epoch 42/100
2000/2000 [==============================] - 1810s 905ms/step - loss: 0.3773 - acc: 0.8325 - val_loss: 0.5178 - val_acc: 0.7750

Epoch 00042: val_loss did not improve from 0.48106
Epoch 43/100
2000/2000 [==============================] - 1857s 929ms/step - loss: 0.3679 - acc: 0.8406 - val_loss: 0.5620 - val_acc: 0.7764

Epoch 00043: val_loss did not improve from 0.48106
Epoch 44/100
2000/2000 [========================

KeyboardInterrupt: 